# GAMA-12 master catalogue: Flags

In [ ]:
import numpy as np
from astropy.table import Table

import itertools

In [ ]:
SUFFIX = "20180218"
FIELD = "GAMA-12"
catname = "../../dmu1/dmu1_ml_GAMA-12/data/master_catalogue_gama-12_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [ ]:
u_bands = [           "KIDS u"]
g_bands = ["DECam g", "KIDS g", "SUPRIME g", "GPC1 g"]
r_bands = ["DECam r", "KIDS r", "SUPRIME r", "GPC1 r"]
i_bands = [           "KIDS i", "SUPRIME i", "GPC1 i"]
z_bands = ["DECam z",           "SUPRIME z", "GPC1 z", "VIRCAM Z"]
y_bands = [                     "SUPRIME y", "GPC1 y", "VIRCAM Y", "WFCAM Y"]
J_bands = [                                            "VIRCAM J", "WFCAM J"]
H_bands = [                                            "VIRCAM H", "WFCAM H"]
K_bands = [                                            "VIRCAM K", "WFCAM K"]

all_bands = [u_bands, g_bands, r_bands, i_bands, z_bands, y_bands, J_bands, H_bands, K_bands]

## 1. Magnitudes and magnitude erros

In [ ]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue[flagcol] = np.zeros(len(master_catalogue), dtype=bool)
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [ ]:
## dmu0: Pan-STARRS forced photometry cat 
gpc1_err = 0.0010860000038519502
bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_{}_ap".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
    mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)
    

### 2.a DECaLS Total magnitudes

In [ ]:
decam_mag = 14.999935
bands = ["DECam g", "DECam r", "DECam z"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    col = "m_{}".format(basecol)
    flagcol = "flag_{}".format(basecol)
    
    mask  = np.where((master_catalogue[col]  == decam_mag) or (master_catalogue[col] < 7))[0]
    
    print('  Total magnitude')
    flag_mag(flagcol, mask)

## 2. Outliers

In [ ]:
def flag_outliers(x, y, xerr, yerr, flagcol_x, flagcol_y, labels=["x", "y"]):
    
    # Add flag columns if does not exist
    if flagcol_x not in master_catalogue.colnames:
        master_catalogue[flagcol_x] = np.zeros(len(master_catalogue), dtype=bool)
    if flagcol_y not in master_catalogue.colnames:
        master_catalogue[flagcol_y] = np.zeros(len(master_catalogue), dtype=bool)
    flagx, flagy = master_catalogue[flagcol_x], master_catalogue[flagcol_y]
        
    ## Find outliers  
    # Use only finite values
    mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(xerr) & np.isfinite(yerr)
    x, y = np.copy(x[mask]), np.copy(y[mask])
    xerr, yerr = np.copy(xerr[mask]), np.copy(yerr[mask])
    flagx, flagy = np.copy(flagx[mask]), np.copy(flagy[mask])
    
    # Set the minimum error to 10^-3
    if (len(xerr)!=0 and len(yerr)!=0):
        np.clip(xerr, 5e-3, np.max(xerr), out=xerr)
        np.clip(yerr, 5e-3, np.max(yerr), out=yerr)

    diff = y - x
    
    x_label, y_label = labels
    
    # If the difference is all NaN there is nothing to compare.
    if np.isnan(diff).all():
        print("No sources have both {} and {} values.".format(
            x_label, y_label))
        print("")
        return
    
    diff_label = "{} - {}".format(y_label, x_label)
    print("{}:".format(diff_label))
    
    # Chi2 (Normalized difference)
    ichi2 = np.power(diff, 2) / (np.power(xerr, 2) + np.power(yerr, 2))
    mask2 = ichi2 != 0.0
    diff, ichi2 = np.copy(diff[mask2]), np.copy(ichi2[mask2])
    x, y, xerr, yerr = np.copy(x[mask2]), np.copy(y[mask2]), np.copy(xerr[mask2]), np.copy(yerr[mask2])

    log_ichi2_25p, log_ichi2_75p = np.percentile(np.log10(ichi2), [25., 75.])
    out_lim = log_ichi2_75p + 3.2*abs(log_ichi2_25p-log_ichi2_75p)
    
    outliers = np.log10(ichi2) > out_lim 
    nb_outliers = len(x[outliers])

    print("  Number of outliers: {}".format(nb_outliers))
    
    
    ## Flag outliers
    flagx[outliers], flagy[outliers] = np.ones(nb_outliers, dtype=bool), np.ones(nb_outliers, dtype=bool)

In [ ]:
for band_of_a_kind in all_bands:
    for band1, band2 in itertools.combinations(band_of_a_kind, 2):

        basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
        
        # Aperture mag
        try:
            col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
            ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
            flagcol1, flagcol2 = "flag_{}_ap".format(basecol1), "flag_{}_ap".format(basecol2)
        
            flag_outliers(master_catalogue[col1], master_catalogue[col2],
                          master_catalogue[ecol1], master_catalogue[ecol2],
                          master_catalogue[flagcol1], master_catalogue[flagcol2],
                          labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
        except KeyError:
            print("Probably no aperture mag on {} or {}".format(col1, col2))
                      
        try:
            # Tot mag
            col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
            ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
            flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
                      
            flag_outliers(master_catalogue[col1], master_catalogue[col2], 
                          master_catalogue[ecol1], master_catalogue[ecol2],
                          master_catalogue[flagcol1], master_catalogue[flagcol2],
                          labels=("{} (total)".format(band1), "{} (total)".format(band2)))
        except KeyError:
            print("Probably no aperture mag on {} or {}".format(col1, col2))            
                      

## 3. Save table

In [ ]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)